In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [87]:

engine = create_engine("sqlite:///./poc.db")  # Or your PostgreSQL URL
df = pd.read_sql("SELECT * FROM jobs", engine)
print(df)

      id                          job
0      1          Marketing Assistant
1      2                     VP Sales
2      3           Biostatistician IV
3      4    Account Representative II
4      5                 VP Marketing
..   ...                          ...
178  179         Software Engineer II
179  180              Statistician IV
180  181         Programmer Analyst I
181  182     Account Representative I
182  183  Administrative Assistant IV

[183 rows x 2 columns]


In [90]:
# Number of employees hired for each job and department in 2021 divided by quarter. The
# table must be ordered alphabetically by department and job.

datefrom = '2021-01-01'
dateto = '2021-12-31'

engine = create_engine("sqlite:///./poc.db")  # Or your PostgreSQL URL

from sqlalchemy import text

# Sqlite does not support UNPIVOT function, but in this way it is simulated
query = text("""
    SELECT department Department, job Job, 
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 1 THEN 1 ELSE 0 END) AS Q1,
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 2 THEN 1 ELSE 0 END) AS Q2,
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 3 THEN 1 ELSE 0 END) AS Q3,
    SUM(CASE WHEN CAST((strftime('%m', datetime) + 2) / 3 AS INTEGER) = 4 THEN 1 ELSE 0 END) AS Q4
    FROM hired_employees e
    LEFT JOIN jobs j
    ON e.job_id = j.id
    LEFT JOIN departments d
    ON e.department_id = d.id
    WHERE datetime >= :datefrom AND datetime <= :dateto
    GROUP BY department, job
    ORDER BY department, job
""")

df = pd.read_sql(query, con=engine, params={
    "datefrom": f"{datefrom}T00:00:00Z",
    "dateto": f"{dateto}T23:59:59Z"
})

display(df)

,Department,Job,Q1,Q2,Q3,Q4
0,Accounting,Account Representative IV,1,0,0,0
1,Accounting,Actuary,0,1,0,0
2,Accounting,Analyst Programmer,0,0,1,0
3,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,Accounting,Cost Accountant,0,1,0,0
...,...,...,...,...,...,...
928,Training,Teacher,0,2,0,0
929,Training,Technical Writer,0,0,1,0
930,Training,VP Product Management,1,0,0,0
931,Training,VP Quality Control,0,1,0,0


In [86]:
# List of ids, name and number of employees hired of each department that hired more
# employees than the mean of employees hired in 2021 for all the departments, ordered
# by the number of employees hired (descending).

datefrom = '2021-01-01'
dateto = '2021-12-31'

engine = create_engine("sqlite:///./poc.db")  # Or your PostgreSQL URL

from sqlalchemy import text

query = text("""
    SELECT d.department Department, count(*) Hired
    FROM hired_employees e
    INNER JOIN (
                SELECT *
                FROM (         
                        SELECT department_id, count(*) hires_dep,
                        AVG(COUNT(*)) OVER () AS avg_hires
                        FROM hired_employees e
                        WHERE datetime >= :datefrom AND datetime <= :dateto
                        GROUP BY department_id
                    )q
                WHERE hires_dep > avg_hires             
             ) deps_hire_over_mean         
    ON deps_hire_over_mean.department_id = e.department_id
    LEFT JOIN departments d
    ON d.id = e.department_id
    GROUP BY e.department_id, d.department
    ORDER BY Hired desc
""")

df = pd.read_sql(query, con=engine, params={
    "datefrom": f"{datefrom}T00:00:00Z",
    "dateto": f"{dateto}T23:59:59Z"
})

display(df)

,Department,Hired
0,Support,248
1,Human Resources,242
2,Engineering,241
3,Services,233
4,Business Development,215
5,Research and Development,173
6,Marketing,164


In [81]:
# List of ids, name and number of employees hired of each department that hired more
# employees than the mean of employees hired in 2021 for all the departments, ordered
# by the number of employees hired (descending).

datefrom = '2021-01-01'
dateto = '2021-12-31'

engine = create_engine("sqlite:///./poc.db")  # Or your PostgreSQL URL

from sqlalchemy import text

query = text("""
    SELECT *
    FROM (         
            SELECT department_id, count(*) hires_dep,
            AVG(COUNT(*)) OVER () AS avg_hires
            FROM hired_employees e
            WHERE datetime >= :datefrom AND datetime <= :dateto
            GROUP BY department_id
        )q
    WHERE hires_dep > avg_hires         
""")

df = pd.read_sql(query, con=engine, params={
    "datefrom": f"{datefrom}T00:00:00Z",
    "dateto": f"{dateto}T23:59:59Z"
})

display(df)

,department_id,hires_dep,avg_hires
0,3,148,136.916667
1,4,185,136.916667
2,5,205,136.916667
3,6,201,136.916667
4,7,200,136.916667
5,8,216,136.916667
6,9,142,136.916667


In [28]:
engine.execute("delete from departments")

In [15]:
import json
import random

jobs = [
    "Data Analyst", "Software Engineer", "Product Manager", "Sales Associate", "HR Specialist",
    "DevOps Engineer", "UX Designer", "Marketing Manager", "Financial Analyst", "IT Support"
]

data = [{"id": i, "job": random.choice(jobs)} for i in range(2000, 2000 + 1002)]